In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Set the backend to Agg to allow image saving without a display window
matplotlib.use("Agg")

def run_stock_analysis(tickers, start_date, end_date):
    """
    Downloads stock data, calculates financial metrics,
    and generates a performance report and chart.
    """
    print(f"Fetching data for: {', '.join(tickers)}...")

    # 1. Download Historical Data (Adjusted Close Prices)
    # auto_adjust=True ensures we account for stock splits and dividends
    data = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        auto_adjust=True,
        progress=False
    )['Close']

    if data.empty:
        print("Error: No data found. Please check tickers or your internet connection.")
        return

    # 2. Calculate Daily Returns
    returns = data.pct_change().dropna()

    # 3. Compute Annualized Metrics
    # Annualized Mean Return (252 trading days per year)
    annual_return = returns.mean() * 252
    # Annualized Volatility (Standard Deviation)
    volatility = returns.std() * np.sqrt(252)
    # Sharpe Ratio (Assuming 0% risk-free rate)
    sharpe_ratio = annual_return / volatility

    # 4. Calculate Maximum Drawdown (The peak-to-trough decline)
    cumulative_returns = (1 + returns).cumprod()
    running_max = cumulative_returns.cummax()
    drawdowns = (cumulative_returns - running_max) / running_max
    max_drawdown = drawdowns.min()

    # 5. Create Summary DataFrame
    summary = pd.DataFrame({
        "Annual Return (%)": (annual_return * 100).round(2),
        "Volatility (%)": (volatility * 100).round(2),
        "Sharpe Ratio": sharpe_ratio.round(2),
        "Max Drawdown (%)": (max_drawdown * 100).round(2)
    })

    # 6. Generate Performance Chart
    plt.figure(figsize=(12, 6))
    for ticker in cumulative_returns.columns:
        plt.plot(cumulative_returns.index, cumulative_returns[ticker], label=ticker)

    plt.title(f"Portfolio Performance ({start_date} to {end_date})", fontsize=14)
    plt.xlabel("Date")
    plt.ylabel("Growth of $1 Investment")
    plt.legend(loc="upper left")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Save files to the local environment (Google Colab Files)
    plt.savefig("stock_performance_chart.png", dpi=150)
    summary.to_csv("stock_summary_report.csv")

    print("\n--- Performance Summary Report ---")
    print(summary.sort_values(by="Sharpe Ratio", ascending=False))
    print("\nSuccess: 'stock_performance_chart.png' and 'stock_summary_report.csv' have been generated.")

# --- Execution Block ---
if __name__ == "__main__":
    # You can change these tickers to any valid Yahoo Finance symbols
    selected_tickers = ["TSLA", "JPM", "COST", "LLY"]

    run_stock_analysis(
        tickers=selected_tickers,
        start_date="2023-01-01",
        end_date="2025-01-01"
    )

Fetching data for: TSLA, JPM, COST, LLY...

--- Performance Summary Report ---
        Annual Return (%)  Volatility (%)  Sharpe Ratio  Max Drawdown (%)
Ticker                                                                   
COST                39.03           18.96          2.06            -10.27
JPM                 33.96           22.17          1.53            -13.49
LLY                 42.83           29.57          1.45            -24.13
TSLA                83.09           58.31          1.43            -51.57

Success: 'stock_performance_chart.png' and 'stock_summary_report.csv' have been generated.
